In [1]:
import requests
import pandas as pd
from faKy import process_text_readability

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sandrobarreshamers/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.9/25.9 MB 14.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.8 MB/s eta 0:00:0000:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [39]:
from datasets import load_dataset

# Load the 'liar' dataset
dataset = load_dataset('liar')

# Access the 'train' split
test_data = dataset['test']

# Convert the training data to a Pandas DataFrame
test_df = train_data.to_pandas()

# Now you have the training data in a Pandas DataFrame (train_df)
# You can work with train_df as needed for your analysis


In [40]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1283 entries, 0 to 1282
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1283 non-null   object 
 1   label                 1283 non-null   int64  
 2   statement             1283 non-null   object 
 3   subject               1283 non-null   object 
 4   speaker               1283 non-null   object 
 5   job_title             1283 non-null   object 
 6   state_info            1283 non-null   object 
 7   party_affiliation     1283 non-null   object 
 8   barely_true_counts    1283 non-null   float32
 9   false_counts          1283 non-null   float32
 10  half_true_counts      1283 non-null   float32
 11  mostly_true_counts    1283 non-null   float32
 12  pants_on_fire_counts  1283 non-null   float32
 13  context               1283 non-null   object 
dtypes: float32(5), int64(1), object(8)
memory usage: 115.4+ KB


In [41]:
test_df.head()

,id,label,statement,subject,speaker,job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,11972.json,3,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30.0,30.0,42.0,23.0,18.0,Radio interview
1,11685.json,0,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2.0,1.0,0.0,0.0,0.0,a news conference
2,11096.json,0,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63.0,114.0,51.0,37.0,61.0,comments on ABC's This Week.
3,5209.json,1,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1.0,1.0,3.0,1.0,1.0,a radio show
4,9524.json,5,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,,Wisconsin,democrat,5.0,7.0,2.0,2.0,7.0,a web video


In [42]:
print(test_df.columns)

Index(['id', 'label', 'statement', 'subject', 'speaker', 'job_title',
       'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts',
       'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts',
       'context'],
      dtype='object')


In [43]:
test_df = test_df[['id', 'label','statement']].copy()
test_df.head()

,id,label,statement
0,11972.json,3,Building a wall on the U.S.-Mexico border will...
1,11685.json,0,Wisconsin is on pace to double the number of l...
2,11096.json,0,Says John McCain has done nothing to help the ...
3,5209.json,1,Suzanne Bonamici supports a plan that will cut...
4,9524.json,5,When asked by a reporter whether hes at the ce...


In [44]:
def condition(x):
    if x == 0:
        return 1
    elif x== 1:
        return 2
    elif x== 2:
        return 2
    elif x== 3:
        return 0
    elif x == 4:
        return 2
    elif x == 5:
        return 1




In [45]:
test_df['binary_label'] = test_df['label'].apply(condition)
test_df = test_df[['id', 'binary_label','statement']].copy()
test_df.head()

,id,binary_label,statement
0,11972.json,0,Building a wall on the U.S.-Mexico border will...
1,11685.json,1,Wisconsin is on pace to double the number of l...
2,11096.json,1,Says John McCain has done nothing to help the ...
3,5209.json,2,Suzanne Bonamici supports a plan that will cut...
4,9524.json,1,When asked by a reporter whether hes at the ce...


In [46]:
value_counts = test_df['binary_label'].value_counts()

# Display the count of unique values
print(value_counts)

2    730
1    342
0    211
Name: binary_label, dtype: int64


In [47]:
from faKy import process_text_readability
test_df['readability'] = test_df['statement'].apply(process_text_readability)

In [50]:
from faKy import process_text_complexity
test_df['compressed_size'] = test_df['statement'].apply(process_text_complexity)

In [53]:
from faKy import process_text_vader
test_df[['vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']] = test_df['statement'].apply(
                                                                        lambda x: pd.Series(process_text_vader(x)))



In [55]:
from faKy import count_named_entities, count_ner_labels, create_input_vector_NER, ner_labels

test_df['tot_ner_count'] = test_df['statement'].apply(count_named_entities)
test_df['ner_counts'] = test_df['statement'].apply(count_ner_labels)
test_df['input_vector_ner'] = test_df['ner_counts'].apply(create_input_vector_NER)

for tag in ner_labels:
    col_name = f'NER_{tag}'
    test_df[col_name] = test_df['input_vector_ner'].apply(
                                    lambda x: x[ner_labels.index(tag)] if tag in ner_labels else 0)


In [57]:
from faKy import count_pos, create_input_vector_pos, pos_tags
test_df['pos counts'] = test_df['statement'].apply(count_pos)
test_df['input_vector_pos'] = test_df['pos counts'].apply(create_input_vector_pos)
for tag in pos_tags:
    col_name = f'pos_{tag}'
    test_df[col_name] = test_df['input_vector_pos'].apply(lambda x: x[pos_tags.index(tag)] if tag in pos_tags else 0)



In [61]:
test_df.head()

,id,binary_label,statement,readability,compressed_size,vader_neg,vader_neu,vader_pos,vader_compound,tot_ner_count,...,pos_NOUN,pos_NUM,pos_PART,pos_PRON,pos_PROPN,pos_PUNCT,pos_SCONJ,pos_SYM,pos_VERB,pos_X
0,11972.json,0,Building a wall on the U.S.-Mexico border will...,49.542727,5197,0.000,1.000,0.000,0.0000,1,...,3.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0
1,11685.json,1,Wisconsin is on pace to double the number of l...,81.855000,5548,0.000,0.894,0.106,0.0772,2,...,4.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2,11096.json,1,Says John McCain has done nothing to help the ...,103.625000,4816,0.201,0.799,0.000,-0.3089,1,...,1.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,3.0,0.0
3,5209.json,2,Suzanne Bonamici supports a plan that will cut...,43.963077,6020,0.127,0.602,0.271,0.3400,2,...,3.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,3.0,0.0
4,9524.json,1,When asked by a reporter whether hes at the ce...,62.625000,10724,0.225,0.683,0.092,-0.5994,1,...,5.0,0.0,1.0,1.0,3.0,2.0,1.0,0.0,4.0,0.0


In [65]:
import os

# Get the directory of the currently running script or Jupyter Notebook
current_directory = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()

# Define the file name
file_name = "test_data.csv"

# Construct the full file path
file_path = os.path.join(current_directory, file_name)

# Save the DataFrame to the CSV file
test_df.to_csv(file_path, index=True)


In [ ]:
test_df.h